In [1]:
from faker import Faker
import random
import pandas as pd
from datetime import datetime, timedelta

# Generate 100 users with detailed location data for a day
fake = Faker()
data = []

for _ in range(100):
    user_id = fake.uuid4()
    user_name = fake.name()
    start_time = datetime(2024, 1, 1, 8, 0, 0)  # Starting at 8:00 AM
    end_time = datetime(2024, 1, 1, 20, 0, 0)  # Ending at 8:00 PM

    while start_time < end_time:
        data.append({
            'User_ID': user_id,
            'User_Name': user_name,
            'Timestamp': start_time,
            'Latitude': fake.latitude(),
            'Longitude': fake.longitude(),
            'Location': random.choice(['Park', 'Restaurant', 'Office', 'Home', 'Shopping Mall'])
        })
        start_time += timedelta(minutes=random.randint(15, 120))  # Random time interval between 15 to 120 minutes

# Save the generated data to a CSV file
df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta

# Read the dataset
df = pd.read_csv('data.csv')

# Convert categorical labels to numerical values
le = LabelEncoder()
df['Location'] = le.fit_transform(df['Location'])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[['Latitude', 'Longitude']], df['Location'], test_size=0.2, random_state=42)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Function to predict next location and time for a given user name
def predict_next_location_and_time(user_name):
    user_data = df[df['User_Name'] == user_name][['Latitude', 'Longitude']]
    
    if user_data.empty:
        return None, None  # User name not found in the dataset
    
    predicted_location = clf.predict(user_data)
    predicted_location_label = le.inverse_transform(predicted_location)[0]
    
    # Assuming the current time is the prediction time
    prediction_time = datetime.now() + timedelta(minutes=15)  # Adjust this based on your use case
    
    return predicted_location_label, prediction_time

# User input for the user name
user_name_input = input("Enter User Name: ")

# Example of predicting the next location and time for the user name entered by the user
predicted_location, prediction_time = predict_next_location_and_time(user_name_input)

if predicted_location is not None:
    print(f'Predicted Next Location: {predicted_location}')
    print(f'Prediction Time: {prediction_time}')
else:
    print(f'User Name not found in the dataset.')


Enter User Name:  Stacey Woods


Predicted Next Location: Restaurant
Prediction Time: 2024-01-30 12:42:25.322326
